### 1.1

The transition probability matrix A is constructed to model the robot’s motion within the grid-based environment under strict physical constraints. Transitions are allowed only between neighboring cells in the four cardinal directions, while diagonal movements and transitions through obstacles are explicitly excluded. For each valid state, all feasible outgoing transitions toward accessible neighboring states are assigned equal probability, ensuring a uniform redistribution of probability mass among admissible moves. This construction guarantees that the resulting matrix is stochastic and accurately reflects the local and non-deterministic nature of the robot’s motion within the maze.

The transition matrix A represents the motion dynamics of the robot within the maze by encoding, for each state, the probability of transitioning to any other state in a single time step. Each row of the matrix corresponds to a starting state, while each column represents a possible destination state. Nonzero entries appear only for physically reachable neighboring cells, indicating that the robot can move exclusively toward accessible adjacent positions. The equality of the transition probabilities reflects the assumption of a uniform random choice among all feasible movements.

The transition matrix $A$ defines the probability of moving from one hidden state to another in the Hidden Markov Model. Each element $A_{ij}$ represents the probability of transitioning from state $i$ to state $j$ in a single time step:

$$
A_{ij} = P(X_{t+1} = j \mid X_t = i)
$$

where $X_t$ is the hidden state at time $t$. In our grid-based environment, transitions are allowed only to physically adjacent and accessible cells (North, South, East, West), while diagonal moves and moves through obstacles are forbidden.

If state $i$ has $n_i$ valid neighboring cells, each feasible transition has equal probability:

$$
A_{ij} =
\begin{cases} 
\dfrac{1}{n_i}, & \text{if state } j \text{ is a valid neighbor of } i \\
0, & \text{otherwise}
\end{cases}
$$




humanize

The transition probability matrix A models the motion of the robot inside the grid-based environment under strict physical constraints. You may transition only between neighboring cells in the four cardinal directions. You may not move diagonally, or through any obstacles. At each valid state, every feasible exit transition towards reachable neighbours is given equal probability, ensuring uniform redistribution of probability mass among admissible moves. With this construction the matrix will be stochastic and will model the local and non deterministic behaviour of the robot in maze.

The matrix A reflects the motion dynamics of the robot in the maze. For each state, it tells us the probability of going to any of the states in one time-step. A starting state is represented by a row in the matrix while each column corresponds to a destination state. According to the entries of the matrix, the robot can only move to its physically reachable neighbouring cells. The transition probabilities being equal reflects that we assume uniform random choice over all feasible movements.

The transition matrix $A$ indicates the probability of transitioning from one hidden state to another in the Hidden Markov Model. Each element $A_{ij}$ is the probability of going from state $i$ to state $j$ in one time step.

$$
A_{ij} = P(X_{t+1} = j \mid X_t = i).
$$

Here, the hidden state in time $t$ is $X_t$. In a grid world, actors can only transition to adjacent and accessible non-obstructed cells (North, South, East, West). Diagonal moves and moves through obstructed cells are not allowed.

If state i has n_i valid neighboring cells then any valid transition is equally likely.

$$
A_{ij} =
\begin{cases} 
\dfrac{1}{n_i}, & \text{if state } j \text{ is a valid neighbor of } i \\
0, & \text{otherwise}
\end{cases}
$$

### 1.2

The emission probability matrix encodes the relationship between the robot’s position and its observations. In each position of the grid, the robot identifies whether the neighboring positions North, South, East, West are accessible (i.e. are free). Every possible observation defined in variable `vocabulary` corresponds to a specific combination of available directions. The matrix B is constructed such that for a given state, the column corresponding to the matching observation is assigned a probability of one, while all other entries are zero. The deterministic assignment assumes that the robot’s sensors totally detect the local configuration of free and blocked neighbouring cells.

An emission matrix $B$ provides the probability of obtaining a certain reading from a sensor given the robot’s state. Every element $B_{ik}$ indicates the probability for observing $O_k$ when the robot is in state $i$.

$$
B_{ik} = P(O_t = k \mid X_t = i)
$$

where the observation $O_t$ happened at the time $t$. The robot checks if the neighboring cells in the North, South, East,  West are free. Any combination of accessible paths pertains to a predetermined observation in the `vocabulary`.

The matrix is deterministic:

$$
B_{ik} =
\begin{cases} 
1, & \text{if observation } k \text{ matches the accessible neighbors of state } i \\
0, & \text{otherwise}
\end{cases}
$$

This reflects perfect sensing: the robot can fully detect which neighboring cells are free.

### 1.3

In this snippet, we sample and visualize a sequence of hidden states $Z$ and observations $Y$ of length $T=20$ from the Hidden Markov Model defined earlier. The starting state $Z_0$ is from the uniform distribution $\pi$ over all admissible states, meaning it would be uniformly random. For $t>0$, the states $Z_t$ are obtained from the transition matrix $A$, which gives the probabilities with which the state moves into an accessible neighboring cell:

$$
Z_0 \sim \pi, \quad
Z_t \sim P(X_t \mid X_{t-1}) = A_{Z_{t-1}}
$$

For every hidden state $Z_t$, the observation $Y_t$ is locked in by the emission matrix $B$. This matrix captures the robot’s immediate sensory input—what adjacent cells are free:

$$
Y_t = O_k \quad \text{such that } B_{Z_t, k} = 1
$$

Here’s the kicker: this setup creates sequences $(Z, Y)$ that align perfectly with the model’s rules. The robot only moves where it can, and at each step, it sees exactly which nearby cells are open.


### likelihood

We use the Forward algorithm to compute the likelihood of an observed sequence $Y$. It sums over all possible hidden state sequences efficiently.

Let $\alpha_t(i)$ represent the forward probability of observing the first $t$ observations and ending in hidden state $i$ at time $t$:

$$
\alpha_0(i) = \pi_i \, B_{iY_0}
$$

$$
\alpha_t(j) = \sum_{i=1}^{N} \alpha_{t-1}(i) \, A_{ij} \, B_{jY_t}, \quad t = 1, \dots, T-1
$$

Here, $A_{ij}$ is the transition probability from state $i$ to $j$, $B_{jY_t}$ is the emission probability of observation $Y_t$ given state $j$, and $\pi_i$ is the initial state probability.

The likelihood of the observation sequence $Y$ is then obtained by summing the forward probabilities at the final time step:

$$
P(Y) = \sum_{i=1}^{N} \alpha_{T-1}(i)
$$

This approach avoids enumerating all possible state sequences, providing an efficient way to compute $P(Y)$ for a Hidden Markov Model.


### decoding

The Viterbi algorithm helps us figure out the most likely sequence of hidden states in a Hidden Markov Model (HMM). Here's how it works:

We start by defining $\delta_t(i)$, which represents the highest probability of a state sequence ending in hidden state $i$ at time $t$ that explains the first $t$ observations. Initially, it's just the probability of the first observation given the starting state:

$$
\delta_0(i) = \pi_i \, B_{iY_0}
$$

Then, for each subsequent time step, we calculate:

$$
\delta_t(j) = \max_{i=1,\dots,N} \left[ \delta_{t-1}(i) \, A_{ij} \right] B_{jY_t}, \quad t = 1, \dots, T-1
$$

As we go, we keep track of the best path to each state with backpointers:

$$
\psi_t(j) = \arg\max_{i=1,\dots,N} \left[ \delta_{t-1}(i) \, A_{ij} \right]
$$

Once we've computed all these values, we can find the most probable state sequence. We start at the end and work our way back:

$$
Z_{\text{vit}}[T-1] = \arg\max_i \delta_{T-1}(i)
$$

$$
Z_{\text{vit}}[t] = \psi_{t+1}(Z_{\text{vit}}[t+1]), \quad t = T-2, \dots, 0
$$

This gives us the single best sequence of hidden states that explains the observed data according to the HMM.

The Viterbi algorithm is used to determine the most likely sequence of hidden states given an observation sequence $Y = (y_1, \dots, y_T)$.
To calculate the optimal sequence, we must calculate the probability of being in each state and the maximum one for each t steps. Viterbi checks matrix A and matrix B for applicable matrices as per the following formula.

Initialization (for each state $i$).

$$
\delta_1(i) = \pi_i\, b_i(y_1)
$$

Recursion (for each time $t = 2, \dots, T$ and state $j$).

$$
\delta_t(j) = \max_{i=1}^N \left[ \delta_{t-1}(i)\, a_{ij} \right] b_j(y_t)
$$

Backtracking.

$$
q_T^* = \arg\max_{i=1}^N \delta_T(i)
$$

$$
q_t^* = \psi_{t+1}(q_{t+1}^*), \quad t = T-1, \dots, 1
$$

where $\pi_i$ is the initial probability of being in state $i$, $a_{ij}$ is the $(i, j)$ entry of the transition matrix $A$, and $b_j(y_t)$ is the probability of observing $y_t$ in state $j$, as defined by the emission matrix $B$.

The Viterbi algorithm is used to compute the most probable hidden state sequence for an observation sequence $Y = (y_1, y_2, y_T)$.
To find the best sequence, we need to find the probability of being in each state. We also want the max one for each t steps. Viterbi verifies both matrix A and matrix B for applicable matrices as per the following formula.

Initialization (for each state $i$).

$$
\delta_1(i) = \pi_i\, b_i(y_1)
$$
Recursion (for each time $t = 2, \dots, T$ and state $j$).

$$
\delta_t(j) = \max_{i=1}^N \left[ \delta_{t-1}(i)\, a_{ij} \right] b_j(y_t)
$$

Backtracking.

$$
q_T^* = \arg\max_{i=1}^N \delta_T(i)
$$

$$
q_t^* = \psi_{t+1}(q_{t+1}^*), \quad t = T-1, \dots, 1
$$

where $\pi_i$ is the initial probability of being in state $i$, $a_{ij}$ is the $(i, j)$ entry of the transition matrix $A$, and $b_j(y_t)$ is the probability of observing $y_t$ in state $j$, as defined by the emission matrix $B$.